In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Excel Writer Template for price List process

In [17]:
from pathlib import Path
import shutil
import xlwings as xw
import pandas as pd

def write_df_to_listobject(
    template_path: str | Path,
    output_path: str | Path,
    sheet_name: str,
    table_name: str,
    df: pd.DataFrame,
    *,
    write_header: bool = False  # your table already has headers
):
    # Work on a copy so the original template stays pristine
    template_path = Path(template_path)
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    shutil.copyfile(template_path, output_path)

    app = xw.App(visible=False, add_book=False)
    app.screen_updating = False
    app.display_alerts = False

    try:
        wb = app.books.open(str(output_path))
        sht = wb.sheets[sheet_name]

        # COM ListObject
        lo = sht.api.ListObjects(table_name)

        # Top-left header cell (COM)
        hdr_tl_com = lo.HeaderRowRange.Cells(1, 1)
        hdr_row = hdr_tl_com.Row
        hdr_col = hdr_tl_com.Column

        # xlwings Range from coordinates (xlwings wants row/col, not COM obj)
        hdr_tl = sht.range((hdr_row, hdr_col))

        # 1) write header if requested (usually False when using a real table)
        if write_header:
            hdr_tl.value = list(df.columns)

        # 2) write body in one shot
        body_row = hdr_row + 1 if write_header else hdr_row + 1  # table has a header row
        body_tl = sht.range((body_row, hdr_col))
        body_tl.options(index=False, header=False).value = df

        # 3) resize table to header+body
        n_rows, n_cols = df.shape
        last_cell = sht.range((body_row + n_rows - 1, hdr_col + n_cols - 1))
        full_range = sht.range((hdr_row, hdr_col), last_cell)
        lo.Resize(full_range.api)

        wb.save()
        wb.close()
    finally:
        app.display_alerts = True
        app.screen_updating = True
        app.quit()


In [ ]:
import numpy as np
cols = [f"c{i:02d}" for i in range(16)]
df_main = pd.DataFrame(np.random.randint(0, 1000, size=(900, 16)), columns=cols)

write_df_to_listobject(
    template_path=r"",  # has a sheet "Main" and table "MainTbl"
    output_path=r"",
    sheet_name="Price List",
    table_name="PriceList",
    df=df_main,
    write_header=False
)



In [ ]:
import xlwings as xw
import shutil
template = Path(r"")
out_path = Path(r"")
copy = shutil.copy2(template, out_path)

app = xw.App(visible=True, add_book=False)
wb = app.books.open(copy)

ws = wb.sheets["Price List"]
r = ws.range("A3")
r.api.Interior.ColorIndex = 6

POS Yaml Testing

In [19]:
from config.paths import POS_YAML
from utils.yaml_loader import load_yaml

read_cfg = load_yaml(POS_YAML)

for file in read_cfg["files"]:
    print(file)



{'match_regex': 'accu tech', 'pandas': {'usecols': ['CUST NAME', 'BILLING ZIP', 'SHIP TO CITY', 'SHIP TO STATE', 'SHIP TO ZIP', 'VENDOR ITEM .', 'EXTND QTY', 'ENTND AVG COST']}, 'rename': {'SoldToName': 'CUST NAME', 'PiiPartNumber': 'VENDOR ITEM .', 'ShipQuantity': 'EXTND QTY', 'ExtendedSales': 'ENTND AVG COST', 'BillToCustomerZip': 'BILLING ZIP', 'ShipToState': 'SHIP TO STATE', 'ShipToZip': 'SHIP TO CITY'}}
{'match_regex': 'adi', 'pandas': {'usecols': ['VENDOR PART NUMBER', 'UNITS', 'COST OF SALES', 'SHIP TO ZIPCODE']}, 'rename': {'PiiPartNumber': 'VENDOR PART NUMBER', 'ShipQuantity': 'UNITS', 'ExtendedSales': 'COST OF SALES', 'ShipToZip': 'SHIP TO ZIPCODE'}}
{'match_regex': 'exertis almo', 'pandas': {'usecols': ['NAME', 'MFG PART', 'QUANTITY SHIPPED', 'EXTENDED COST', 'CUSTOMER ZIP', 'CUSTOMER STATE', 'END USER SHIP TO STATE', 'END USER SHIP TO ZIP']}, 'rename': {'SoldToName': 'NAME', 'PiiPartNumber': 'MFG PART', 'ShipQuantity': 'QUANTITY SHIPPED', 'ExtendedSales': 'EXTENDED COST', '